In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import celerite as ce
from celerite import terms
from celerite.modeling import Model

#defining helper functions for model
def expf(t,a,b):
    return np.exp(a*(t-b))

def linef(t, x1, y1, x2, y2):
    m = (y2-y1)/(x2-x1)
    b = y2 - (m*x2)
    return ((m*t)+b)

#defining the model for our fit
class PWModel(Model):
    parameter_names = ("xl", "xr", "al", "bl", "ar", "br")
    
    def get_value(self, t):
        result = np.empty(len(t))
        for i in range(len(t)): #had to tweak this to accept t-arrays, may affect speed...
            if(t[i]<self.xl):
                result[i] = expf(t[i], self.al, self.bl)
            elif(self.xl<t[i] and t[i]<=self.xr):
                result[i] = linef(t[i], self.xl, expf(self.xl, self.al, self.bl), self.xr, expf(self.xr, self.ar, self.br))
            elif(self.xr<t[i]):
                result[i] = expf(t[i], self.ar, self.br)
        return result
    
    #the gradient terms were manually calculated
    def compute_gradient(self, t):
        yl = np.exp(self.al*(self.xl-self.bl))
        yr = np.exp(self.ar*(self.xr-self.br))
        result = np.empty([len(t), 6])
        result2 = np.empty([6, len(t)])
        for i in range(len(t)):
            ylt = np.exp(self.al*(t[i]-self.bl))
            yrt = np.exp(self.ar*(t[i]-self.br))
            if(t[i]<self.xl):
                dxl = 0.
                dxr = 0.
                dal = (t[i]-self.bl) * ylt
                dbl = -1* self.al * ylt
                dar = 0.
                dbr = 0.
                result[i] = np.array([dxl, dxr, dal, dbl, dar, dbr])
                result2[:,i] = result[i]

            elif(self.xl<=t[i] and t[i]<=self.xr):
                dxl = ((t[i]-self.xr)/((self.xr-self.xl)**2)) * (yr + (self.al * self.xl - self.al* self.xr +1) * yl)
                dxr = (((t[i]-self.xl)*(yl-yr) - self.ar*(self.xr-self.xl)*(self.xr-t[i])*yr)/((self.xl-self.xr)**2)) + self.ar * yr
                dal = -1.*((self.xl)/(self.xr-self.xl)) * yl
                dbl = ((self.al)/(self.xr-self.xl))*yl
                dar = ((1/(self.xr-self.xl))+1)*self.xr*yr
                dbr = -1.*((1/(self.xr-self.xl))+1) *self.ar*yr
                result[i] = np.array([dxl, dxr, dal, dbl, dar, dbr])
                result2[:,i] = result[i]
        

            elif(self.xr<t[i]):
                dxl = 0.
                dxr = 0.
                dal = 0.
                dar = (t[i]-self.br) * yrt
                dbl = 0.
                dbr = -1 * self.ar * yrt
                result[i] = np.array([dxl, dxr, dal, dbl, dar, dbr])
                result2[:,i] = result[i]

        return result2

#previously used paramter estimation
params0 = [1100, 1500, 1.3e-2, 100, -9e-3, 2900]
PW_Model = PWModel(xl = params0[0], xr = params0[1], al = params0[2], bl = params0[3], ar = params0[4], br = params0[5])
params_vec = PW_Model.get_parameter_vector()

data1 = "/Users/chris/Documents/QPP/SolarFlareGPs/data/121022782_ctime_lc.txt"
t1, I1 = np.loadtxt(data1, unpack=True)

x = t1
y = I1
yerr = np.sqrt(10000 + I1)

#values from prior work 
A0 = 44.48325411e+07
tau0 = 1.36841695e+00

kernel = terms.RealTerm(log_a = np.log(A0), log_c = np.log(1./tau0))
gp = ce.GP(kernel, mean=PW_Model, fit_mean=True)
gp.compute(x, yerr)

#defining cost function:
def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

def grad_neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.grad_log_likelihood(y)[1]

#fitting
initial_params = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
soln = minimize(neg_log_like, initial_params, jac=grad_neg_log_like, method ="L-BFGS-B", bounds = bounds, args = (y, gp))
gp.set_parameter_vector(soln.x)
print soln
print("Final log-likelihood = {0}".format(-soln.fun))

#max prediction
t = np.linspace(x[0], x[-1:][0], 500)
mu, var = gp.predict(y, t, return_var=True)
std = np.sqrt(var)

#plot data
color = "#ff7f0e"
plt.errorbar(x, y, yerr=yerr, fmt="-k", capsize=0)
plt.plot(t, mu, color=color)
plt.fill_between(t, mu+std, mu-std, color=color, alpha=0.3, edgecolor="none")
plt.ylabel(r"$y$")
plt.xlabel(r"$t$")
plt.xlim(600, 1800)
plt.gca().yaxis.set_major_locator(plt.MaxNLocator(5))
plt.title("maximum likelihood prediction")
plt.show()


















/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in exp
/usr/